## Lineage

This notebook demonstrates how recreate lineages published in the paper [Live imaging of remyelination in the adult mouse corpus callosum]() and available at [idr0113-bottes-opcclones](https://idr.openmicroscopy.org/search/?query=Name:idr0113)

For the specified image, we create the lineage.
To load the data from IDR, we use:
* the Python API
* the JSON API

## Collect the parameter 

In [54]:
imageId = 6001243

## Load the libraries 

In [ ]:
import graphviz
import math
import requests
import pandas as pd

from os.path import expanduser

from idr import connection

##  Load data 

#### Connect to IDR

In [ ]:
conn = connection('idr.openmicroscopy.org')

#### Load the plane information
First load information about the image so we can identify the number of days after induction.

In [77]:
times = {}

image = conn.getObject("Image", imageId)
pixels_id = image.getPrimaryPixels().getId()
query = "from PlaneInfo as Info where pixels.id='" + str(pixels_id) + "'"
infos = conn.getQueryService().findAllByQuery(query, None)
for i in infos:
    times.update({i.theT.getValue(): i.deltaT.getValue()})

### Set up where to query and session
We now use the JSON API to load the Regions of Interest (ROI) and the table with the lineage information

In [95]:
INDEX_PAGE = "https://idr.openmicroscopy.org/webclient/?experimenter=-1"
ROI_URL = "https://idr.openmicroscopy.org/api/v0/m/images/{key}/rois/"
TABLE_URL = "https://idr.openmicroscopy.org/webgateway/table/Image/{key}/query/?query=*"

In [56]:
# create http session
with requests.Session() as session:
    request = requests.Request('GET', INDEX_PAGE)
    prepped = session.prepare_request(request)
    response = session.send(prepped)
    if response.status_code != 200:
        response.raise_for_status()

### Load the Regions of Interest linked to the image
Each ROI has only one shape so we map the timepoint the shape is to the value in days.

In [79]:
qs = {'key': imageId}
url = ROI_URL.format(**qs)
json_data = session.get(url).json()

roi_time_map = {}
# parse the json
for d in json_data['data']:
    roi_id = d['@id']
    for s in d["shapes"]:
        roi_time_map.update({roi_id: times.get(s['TheT'])}) 

### Load the lineage table
Each image with ROIs has a table linked to it capturing the lineage.
We load the table and convert it into a Pandas dataframe.

In [96]:
url = TABLE_URL.format(**qs)
json_data = session.get(url).json()

In [123]:
df = pd.DataFrame(columns=json_data['data']['columns'])
for r in json_data['data']['rows']:
    df.loc[len(df)] = r

In [5]:
df

,Roi,Cell Type,Cell Uncertainty,Lineage Roi,Mother Roi,Mother Uncertainty,Sister Roi,Sister Uncertainty,Cell Death,Roi Name
0,2436052,OPC,uncertain,NaN,NaN,certain,NaN,certain,NaN,1-1_1_0_na_0_1_na_1
1,2436053,OPC,certain,NaN,2436052.0,uncertain,2436055.0,uncertain,NaN,2-1_1_1_1-1_1_0_2-2_0
2,2436054,OPC,certain,NaN,2436052.0,uncertain,2436053.0,uncertain,NaN,2-2_1_2_1-1_1_0_2-1_0
3,2436055,OPC,certain,2436054.0,2436052.0,uncertain,NaN,uncertain,NaN,2-2_1_2_1-1_1_0_2-1_0
4,2436056,OPC,certain,NaN,2436053.0,certain,2436065.0,certain,NaN,3-1_1_6_2-1_1_1_3-2_1_0
5,2436057,OPC,certain,2436056.0,2436053.0,certain,NaN,certain,NaN,3-1_1_6_2-1_1_1_3-2_1_0
6,2436058,OPC,certain,2436057.0,2436053.0,certain,NaN,certain,NaN,3-1_1_6_2-1_1_1_3-2_1_0
7,2436059,OPC,certain,2436058.0,2436053.0,certain,NaN,certain,NaN,3-1_1_6_2-1_1_1_3-2_1_0
8,2436060,OPC,certain,2436059.0,2436053.0,certain,NaN,certain,yes,3-1_1_6_2-1_1_1_3-2_1_0
9,2436061,OPC,certain,NaN,2436053.0,certain,2436060.0,certain,NaN,3-2_1_6_2-1_1_1_3-1_1


### Helper methods to generate the graph 

In [6]:
def is_valid(row):
    mother = row["Mother Roi"]
    sister = row["Sister Roi"]
    if math.isnan(mother) and math.isnan(sister):
        return False
    return True

In [7]:
def find_mothers():
    ids = []
    for index, row in df.iterrows():
        mother = row["Mother Roi"]
        if math.isnan(mother):
            continue
        v = str(int(mother))
        if v not in ids:
            ids.append(v)
    return ids 

### Create the lineage graph
* Each node corresponds to an ROI.
* Hover over each node of the graph to display information like incubation time for the node.
* Click on the node to launch the image viewer in IDR
* The format set for export is ``pdf`` (default), if you wish to export as png, svg for example, modify the ``format`` parameter below.

In [42]:
f = graphviz.Digraph(comment='Lineage', format='pdf')
f.attr(rank='sink')
f.attr('node', shape='circle')

 Create the nodes of the graph

In [43]:
# Set various parametes used in the graph
COLOR_OPC = "red"
COLOR_PM = "orange"
COLOR_M = "cyan"
COLOR_DEAD = "purple"
COLOR_UNCERTAIN = "black"
COLOR_EDGE = "black"
STYLE_CERTAIN = "solid"
STYLE_UNCERTAIN = "dotted"

In [44]:
ids = []
# determine the mothers rois
mothers = find_mothers()
for index, row in df.iterrows():
    id = str(row["Roi"])
    if is_valid(row) is False and id not in mothers:
        continue
    type = str(row["Cell Type"])
    ids.append(id)
    dead = "no"
    dead_value = str(row["Cell Death"])
    uncertainty = str(row["Cell Uncertainty"])
    if dead_value != "nan":
        dead = str(dead_value)
    value = "Type: %s\nCertainty: %s\nDeath: %s\nDays After Induction: %s\nROI ID: %s" % (type, uncertainty, dead, roi_time_map.get(id), id)
    color = COLOR_OPC
    width = "0.1"
    if type == "PM":
        color = COLOR_PM
    elif type == "M":
        color = COLOR_M
    if uncertainty == "uncertain":
        color = "%s:%s" % (COLOR_UNCERTAIN, color)
        width = "0.2"
    f.node(id, "", URL='https://idr.openmicroscopy.org/iviewer/?roi=%s' % id, style="radial", color=color, fixedsize='true', width=width, tooltip=value)
    # Create a node indicating that the cell is dead
    if dead_value != "nan":
        nv = "dead_%s" % id
        f.node(nv, "", style='filled', color=COLOR_DEAD, fixedsize='true', width='0.1')

Create the edges

In [45]:
edges = []
for index, row in df.iterrows():
    if is_valid(row) is False:
        continue
    id = str(row["Roi"])
    color = COLOR_EDGE
    lineage = row["Lineage Roi"]
    if math.isnan(lineage) is False:
        f.edge(str(int(lineage)), id, label='', color=color, arrowhead="none")  
    value = row["Mother Roi"]
    if math.isnan(value) is False and math.isnan(lineage) is True:
        certainty = row["Mother Uncertainty"]
        style = STYLE_CERTAIN
        if certainty != "certain":
            style = STYLE_UNCERTAIN
        f.edge(str(int(value)), id, label='', color=color, style=style, arrowhead="none")        
    dead_value = str(row["Cell Death"])
    if dead_value != "nan":
        nv = "dead_%s" % id
        f.edge(id, nv, label='', color=color, arrowhead="none", tooltip="Cell is dead")

Create the legend

In [46]:
with f.subgraph(name='legend') as c:
    c.attr('node', shape='plaintext', fontsize='8')
    c.node("key1", label='''<<table border="0" cellpadding="2" cellspacing="0" cellborder="0">
      <tr><td align="left" port="i1">certain</td></tr>
      <tr><td align="left" port="i2">uncertain</td></tr>
      </table>>''')
    c.node("key2", label='''<<table border="0" cellpadding="2" cellspacing="0" cellborder="0">
      <tr><td align="right" port="i1">&nbsp;</td></tr>
      <tr><td align="right" port="i2">&nbsp;</td></tr>
      </table>>''')
    c.edge("key1:i1", "key2:i1", label='', color=COLOR_EDGE, style=STYLE_CERTAIN, arrowhead="none")
    c.edge("key1:i2", "key2:i2", label='', color=COLOR_EDGE, style=STYLE_UNCERTAIN, arrowhead="none")
    c.node("OPC")
    c.node("PM")
    c.node("M")
    c.node("Cell Death")
    c.node("Uncertain Cell Type")
    c.attr('node', shape='circle', fontsize='6')
    c.node("opc1", "", style='filled', color=COLOR_OPC, fixedsize='true', width='0.1')
    c.node("pm1", "", style='filled', color=COLOR_PM, fixedsize='true', width='0.1')
    c.node("m1", "", style='filled', color=COLOR_M, fixedsize='true', width='0.1')
    c.node("death1", "", style='filled', color=COLOR_DEAD, fixedsize='true', width='0.1')
    c.node("type1", "", style='filled', color=COLOR_UNCERTAIN, fixedsize='true', width='0.1')
    c.edge("OPC", "opc1", label='', style="invis")
    c.edge("PM", "pm1", label='', style="invis")
    c.edge("M", "m1", label='', style="invis")
    c.edge("Cell Death", "death1", label='', style="invis")
    c.edge("Uncertain Cell Type", "type1", label='', style="invis")

Display the graph

In [126]:
f

### Save the lineage
* To save the lineage as PDF (default), run the cell below.
* To save in another format, edit the cell creating the graph i.e.
``f = graphviz.Digraph(comment='Lineage', format='pdf')``
* Replace the ``format`` attribute by ``png`` or ``svg`` for example, and re-run the cells generating the graph.

The lineage is saved in the ``home`` directory, to download it 
* Go to the ``home`` directory
* Select the checkbox next to the file.
* Click ``Download`` in the menu bar that shows up when 

In [125]:
home = expanduser("~")
f.view(home+"/lineage_%s" % imageId)

AttributeError: '_io.TextIOWrapper' object has no attribute 'view'

### License (BSD 2-Clause) 

Copyright (C) 2021 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.